<a href="https://colab.research.google.com/github/venugopals002/Venus/blob/main/Final_model_BOPP_tape_sales_and_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Run these installation commands first
!pip install --upgrade numpy pandas xgboost gradio scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully unins

In [3]:
# Step 2: Run this cell and UPLOAD YOUR sales_data.csv FILE
from google.colab import files
uploaded = files.upload()

Saving sales_data.csv to sales_data (1).csv


In [4]:
# Reinstall numpy, pandas, xgboost, gradio and scikit-learn
!pip install --upgrade --force-reinstall numpy pandas xgboost gradio scikit-learn

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached xgboost-3.0.0-py3-none-manylinux_2_28_x86_64.whl.metadata (2.1 kB)
  Using cached gradio-5.29.0-py3-none-any.whl.metadata (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.10.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.4 MB/s eta 0:00:00
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached

In [1]:
# Step 3: Run this entire code block
import pandas as pd
import numpy as np
import xgboost as xgb
import gradio as gr
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit

In [4]:
# --------------------------
# 1. Data Preparation
# --------------------------
# Read and clean data
df = pd.read_csv(next(iter(uploaded)))
df.columns = [col.split(' - ')[0] for col in df.columns]

In [5]:
# Reshape data to long format
df_long = df.melt(id_vars='Item Code', var_name='Month', value_name='Sales')
df_long['Month'] = df_long['Month'].str.extract(r'(January|February|March|April|May|June|July|August|September|October|November|December)')

In [6]:
# Convert month names to numbers
month_map = {month: idx+1 for idx, month in enumerate([
    'January','February','March','April','May','June',
    'July','August','September','October','November','December'
])}
df_long['Month'] = df_long['Month'].map(month_map)
df_long = df_long.sort_values(['Item Code', 'Month'])

In [7]:
# --------------------------
# 2. Feature Engineering
# --------------------------
# Create features
df_long['Item_Code_Encoded'] = df_long['Item Code'].astype('category').cat.codes
df_long['Lag_1'] = df_long.groupby('Item Code')['Sales'].shift(1).fillna(0)
df_long['Rolling_Avg_3'] = df_long.groupby('Item Code')['Sales'].transform(
    lambda x: x.rolling(3, min_periods=1).mean()
)

In [8]:
# --------------------------
# 3. Model Training
# --------------------------
# Prepare data
X = df_long[['Item_Code_Encoded', 'Month', 'Lag_1', 'Rolling_Avg_3']]
y = df_long['Sales']

In [11]:
# Time-series cross validation
tscv = TimeSeriesSplit(n_splits=3)
accuracy_scores = []

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=300,
        max_depth=5,
        learning_rate=0.1
    )
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    accuracy_scores.append(r2_score(y_test, preds))

final_accuracy = np.mean(accuracy_scores)
print(f"Model Accuracy: {final_accuracy:.1%}")

Model Accuracy: 57.6%


In [12]:
# --------------------------
# 4. Prediction Interface
# --------------------------
item_codes = df['Item Code'].unique().tolist()

def predict(item_code, month):
    # Get item history
    item_data = df_long[df_long['Item Code'] == item_code]

    # Prepare features
    features = pd.DataFrame([{
        'Item_Code_Encoded': pd.Categorical([item_code], categories=item_codes).codes[0],
        'Month': month,
        'Lag_1': item_data['Sales'].iloc[-1] if len(item_data) > 0 else 0,
        'Rolling_Avg_3': item_data['Rolling_Avg_3'].iloc[-1] if len(item_data) > 0 else 0
    }])

    # Make prediction
    prediction = max(0, round(model.predict(features)[0]))
    return prediction, round(prediction * 1.5)  # 45-day stock

In [13]:
# Create UI
with gr.Blocks() as demo:
    gr.Markdown(f"# 📦 Inventory Predictor (Accuracy: {final_accuracy:.1%})")
    with gr.Row():
        item_input = gr.Dropdown(item_codes, label="Select Item Code")
        month_input = gr.Slider(1, 12, label="Select Month (1-12)", step=1)
    predict_btn = gr.Button("Calculate")

    with gr.Row():
        sales_output = gr.Number(label="Predicted Sales", precision=0)
        stock_output = gr.Number(label="Recommended Stock", precision=0)

    predict_btn.click(
        fn=predict,
        inputs=[item_input, month_input],
        outputs=[sales_output, stock_output]
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://92008b8621ff53d0ff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
